# Data-informed parameter synthesis for population pDTMCs, notebook 3

This notebook provides main part of analysis presented in the paper - Data-informed parameter synthesis. It consists of these parts:
1. [Load Parameter synthesis results](#one)
2. [Direct parameter synthesis via Storm and PRISM](#two)

Before running, please add [Z3](https://github.com/Z3Prover/z3) to PATH and check if it runs correctly.

There are three things to setup in this notebook:
1. `agents_quantities` -- set of agents number to be considered - set numbers for which you want to run analysis,
2. `alpha` and `n_samples`  to set the margin of the data -  `margin(alpha, n_samples, data)`,
3. `n` and `epsilon` in `check_deeper(thresh,prop,data,a,b,n,epsilon)` -- thresholds used in parameter space division.

## IMPORTS, PATHS, and SETTINGS

if a package fails to load due to it is not installed please uncomment and use following code with respective package name

In [ ]:
#import sys
#!{sys.executable} -m pip install <package_name>

In [ ]:
import os,sys

In [ ]:
os.getcwd()

In [ ]:
cwd = os.getcwd()
os.chdir("..")
sys.path.append(os.getcwd())
os.chdir(cwd)

Set agents quantities to be examined

In [ ]:
agents_quantities = [2,3,5,10]

<a name="one"></a>

## 1. LOADING OF DATA

### DATA LOADING

The data we are about to load are in a form data points for given N and number of successes, hence they match already loaded f dictionaries. 

They can be seen as a points of the polynomials.

In [ ]:
from src.load import load_all_data

Load data

In [ ]:
D = load_all_data("data/data*.csv")

In [ ]:
D_multiparam = load_all_data("data/multiparametric/data*.csv")

In [ ]:
D2 = load_all_data("data_1/data*.csv")

In [ ]:
D2_multiparam = load_all_data("data_1/multiparametric/data*.csv")

### showing loaded data

An example of function and data for number of agents N=2 and number of successes i=1

In [ ]:
D[2][1] 

Catch data errors (subzero values)

In [ ]:
from src.load import catch_data_error

In [ ]:
catch_data_error(D,0,1) 
catch_data_error(D2,0,1) 
catch_data_error(D_multiparam,0,1) 
catch_data_error(D2_multiparam,0,1) 

Now all data and f are loaded

### margin

margin to infer intervals in which we wan to search

In [ ]:
from src.load import margin

In [ ]:
print(margin(0.95, 100, 0.01))
print(margin(0.99, 100, 0.01))
print(D[2][0]-margin(0.95, 1500, D[2][0]),",",D[2][0]+margin(0.95, 1500, D[2][0]))

EXPERIMENTAL margin

`0.029501395417987873
0.0356291779716225
0.024749946149793683 , 0.055250053850206315`

<a name="two"></a>

## Data-informed parameter synthesis without using precomputed polynomials

Now we are able to call parameter synthesis with the tresholds.

The property looks like this:

`P>data-margin F[BSCC0] & P<data+margin F[BSCC0] & `

`P>data-margin F[BSCC1] & P<data+margin F[BSCC1] &`

`...`

instead of 

`P=? F[BSCC0] `, `P=? F[BSCC1] `, .... 

in the case of our approach

Here we create data dependent properties

In [ ]:
from src.mc_informed import create_data_informed_properties

In [ ]:
alpha = 0.95
n_samples = 100

In [ ]:
#two-param as whole
for i in agents_quantities:
    create_data_informed_properties(i,D,alpha,n_samples,False,False)    

In [ ]:
#two-param threshold-by-threshold
for i in agents_quantities:
    create_data_informed_properties(i,D,alpha,n_samples,False,True)    

In [ ]:
#multiparam-param as whole
for i in agents_quantities:
    create_data_informed_properties(i,D_multiparam,alpha,n_samples,True,False)    

In [ ]:
#multiparam-param threshold-by-threshold
for i in agents_quantities:
    create_data_informed_properties(i,D_multiparam,alpha,n_samples,True,True)    

### Some theory to understand differences


They are practically three ways to compute data-informed parameter synthesis for multiple properties:
1. (***all at once approach***) is the easiest to run as you join separate properties into one using logical operators, in our case conjunction. After that, you have only one property. 

This may fall badly, because in each splitting solver has to answer every property, hence finding a point which satisfies each property and then proving unsatisfiability of the negation is a hard task.

2. (***prop-by-prop approach***) by running each property separately and then joining the results intersection in case of conjunction of formulas and union in case of a disjunction of formulas

Note that conjunction is easy operation on the set of hyperrectangles while the union is not so easy.
Also, this method does go pretty bad when any of the property makes trouble.

3. (***iterative approach***) In this approach we use easy to load on SMT solver by using only 1 property at the time and take the information of the results to next iteration for the second property. By this means we can cut off unsafe parts of the scanned region. 

There are two downfalls of this method:
Storing information for the next iteration. After we have split the space into safe and unsafe regions in iteration *j* not only safe regions have to be stored, but also a white area with unknown result have to be used because we simply don't know. This is a tricky part because the white area is being changed during the computation, while we only add new red and green hyperrectangles to unsafe a safe area, respectively.

The second problem appears after the last iteration. Since the results of this iteration are the safe regions of the last property in a truncated region. This can be avoided by adding all properties in the last iteration, which of course will increase computation load and may even outrank the first approach.

Remark on heuristics: in order to perform best running one should sort the properties in such order that truncated space is decreasing, in another word, you want to exclude as much of space as you can in each iteration, so solving property which provides it as the first is the best move.

Remark for all approaches: All approaches can be computed in parallel into some degree. For example, the easiest to parallel is the second approach. However, in each approach after splitting the space, new regions can be computed separately.

## PRISM (again)

Now we can call prism with data information

In [ ]:
from src.mc_informed import call_data_informed_prism

In [ ]:
alpha = 0.95
n_samples = 100

### two-param as whole

In [ ]:
#two-param as whole
for i in agents_quantities:
    call_data_informed_prism(i,["p","q"],D,alpha,n_samples,False,False)    
    print()
    print()

### two-param prop-by-prop

In [ ]:
call_data_informed_prism(2,["p","q"],D,alpha,n_samples,False,True)

In [ ]:
call_data_informed_prism(3,["p","q"],D,alpha,n_samples,False,True)

In [ ]:
call_data_informed_prism(5,["p","q"],D,alpha,n_samples,False,True)

In [ ]:
call_data_informed_prism(10,["p","q"],D,alpha,n_samples,False,True)

### multi-param as whole

In [ ]:
#multiparametric-param as whole
for population_size in agents_quantities:
    if population_size ==2:
        continue
    parameters=["p"]
    for value in range(population_size-1):
        parameters.append("q"+str(value+1))
        
    call_data_informed_prism(population_size,parameters,D,alpha,n_samples,True,False)    
    print()
    print()

The first command resulted in arround 9minutes with unsat for all regions. The second two did not halt in reasonable time (>15h). Therefore we show only commands.

### multi-param prop-by-prop

In [ ]:
call_data_informed_prism(3,["p","q1","q2"],D,alpha,n_samples,True,True)

In [ ]:
call_data_informed_prism(5,["p","q1","q2","q3","q4"],D,alpha,n_samples,True,True)

In [ ]:
call_data_informed_prism(10,["p","q1","q2","q3","q4","q5","q6","q7","q8","q9"],D,alpha,n_samples,True,True)

Remark: on windows delete `time` from the command and use something else to measure time to proceed.

I could not find any way to run iterative method for PRISM.

## STORM

Second option is to use Storm. Since Storm is complicated to install I assume you want to use Docker version: Here are the commands to be called in order to synthethetise parameters:

In [ ]:
from src.mc_informed import call_storm

In [ ]:
print(alpha)
print(n_samples)
print(D[2][0]+margin(alpha, 3500, 0.04))

Again, if you are going to run following code on windows, please change `start=$SECONDS` and the last line accordingly (using for example `%time%`) or just skip measuring time. Thank you.

In [ ]:
call_storm(2,["p","q"],D,alpha,n_samples,False)

In [ ]:
call_storm(3,["p","q"],D,alpha,n_samples,False)

In [ ]:
call_storm(5,["p","q"],D,alpha,n_samples,False)

In [ ]:
call_storm(10,["p","q"],D,alpha,n_samples,False)

### multiparam

In [ ]:
call_storm(3,["p","q1","q2"],D_multiparam,alpha,n_samples,True)

In [ ]:
call_storm(5,["p","q1","q2","q3","q4"],D_multiparam,alpha,n_samples,True)

In [ ]:
call_storm(10,["p","q1","q2","q3","q4","q5","q6","q7","q8","q9"],D_multiparam,alpha,n_samples,True)

As you can see, the commnads take each threshold of property and run it separately. It is because there was an Error using multiple Probability operator in the single formula.

Since Storm has crashed on multiparam_asynchronous_10.pm using prop-by-prop approach even for the easiest first property it is no meaning in tryin iterative approach an it is time to use SMT Solver, **z3**.
If you have succeeded, or altered the call to make it work, please do not hesitate to contact us. Thank you.

To see our approach see notebooks `sample_visualise` and `analysis`.

<a name="two"></a>